### function decorators
* purpose: a decorator modifies the behaviors of decorated functions
* the common practice is to put decorators in a stand-alone file such as decorators.py & import when needed
    * this way a decorator could be shared across various functions when needed
* see [this tutorial](https://realpython.com/primer-on-python-decorators/) for a detailed discussion

In [ ]:
""" the actual mechanism of function decoration """

# 1) the target function
def say_whee():
    print("Whee!")

# 2) the decorator takes as argument a 'func' name
def my_decorator(func):
    # the decorator can have an inner function (usually called a wrapper) that
    # defines the modification to the behaviors of 'func'
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    # the decorator returns the inner wrapper function
    return wrapper

In [ ]:
# 3) decoration modifies the target function's behavior by the decorator
say_whee = my_decorator(say_whee)

# test by calling the target function
say_whee()

In [ ]:
""" python simplifies the step 3) decoration syntax by @ symbol """

@my_decorator
def say_whee():
    print("Wheel!")

say_whee()

### passing arguments and returning results for the decorated target function
* 1) inner wrapper function in decorator can take arguments -> this facilitates the target function to be allowed to take arguments
* 2) inner wrapper function in decorator can return func -> this allows the decorated target function to be able to return its results

In [ ]:
""" 1) inner wrapper function can take arguments """

def my_decorator(func):
    # use *args and **kwargs to allow arbitrary arguments
    def wrapper(*args, **kwargs):
        print('somthing before')
        func(*args, **kwargs)
        print('something after')

    return wrapper

@my_decorator
def say_name(name):
    print(name)

say_name('baihuaxie')

In [ ]:
""" 2) inner wrapper function can return func """

def my_decorator(func):
    def wrapper(*args, **kwargs):
        print('something before')
        func(*args, **kwargs)
        print('something after')
        return func(*args, **kwargs)

    return wrapper

@my_decorator
def return_name(name):
    print('greeting!')
    return 'hi, {}'.format(name)

a = return_name('baihuaxie')
print(a)

### introspection
* a python function is allowed to return attributes about itself during runtime
* to maintain the ability to do so for decorated functions, need to decorate the inner wrapper function by @functools.wraps(func)

In [ ]:
""" maintain introspection """

def say_whee():
    print('wheel')
print(say_whee.__name__)

# decoration makes the introspection fail
@my_decorator
def say_whee():
    print('wheel')
print(say_whee.__name__)


# decorate the inner wrapper function by @functools.wraps() would retain introspection on target function
import functools

def my_decorator_B(func):
    """ a generic boilerplater decorator syntax """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('something before')
        value = func(*args, **kwargs)
        print('something after')
        return value
    return wrapper

@my_decorator_B
def say_whee():
    print('wheel')
print(say_whee.__name__)

### examples of decorator use cases:
* 1) count the time elapsed during a function call
* 2) debug codes by printing arguments and function returns for each call
* 3) register methods or plug-in's

In [ ]:
""" 1) timing a function call """
import functools
import time

def timer(func):
    """
    print the runtime of decorated function
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print("{} finished in {} seconds".format(func.__name__, run_time))
        return value
    return wrapper

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum(i**2 for i in range(10000))

waste_some_time(100)


In [ ]:
""" 2) debug codes """
import functools

def debug(func):
    """
    Print the function arguments and return
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # put all arguments in a list by repr()
        args_repr = [repr(a) for a in args]
        # put all keyword arguments in a list
        kwargs_repr = ['{}={}'.format(k, v) for k, v in kwargs.items()]
        signature = ','.join(args_repr + kwargs_repr)
        print('Calling {}({})'.format(func.__name__, signature))
        value = func(*args, **kwargs)
        print('{} returned {}'.format(func.__name__, value))
        return value
    return wrapper

import math

# decorate standard lib (or any pre-defined) functions in this way
math.factorial = debug(math.factorial)

def approximate_e(terms=10):
    """ approximate the natural logarithm constant """
    return sum(1 / math.factorial(n) for n in range(terms))

approximate_e(7)


In [ ]:
""" register methods """

PLUGINS = dict()

# note that there is no need to do an inner wrapper function definition
# as for register decorator the original target function is returned unmodified
def register(func):
    """
    Register func as a method in a registry (a dictionary)
    """
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name):
    return 'hello {}'.format(name)

@register
def say_awesome(name):
    return '{} is awesome'.format(name)

PLUGINS

### decorators accepting arguments
* need a 3-level definition of decorator function

In [ ]:
""" decorator that takes arguments """

import functools

# 1st-level: the outer decorator definition takes as arguments the arguments that
# would be passed to the decorator when used on target functions
def repeat(num_times):
    """ repeat the decorated target function by num_times times """
    # 2nd-level: the intermediate inner definition would take as argument the func
    # that is the target function; from this would look like the original decorator
    # definition without taking arguments
    def decorator_repeat(func):
        # 3rd-level: the inner-most definition is just the wrapper function as before
        # note that this is only needed if the decorator needs to modify target function's behaviors
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for idx in range(num_times):
                print('repeating {}'.format(idx))
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

@repeat(num_times=4)
def greet(name):
    print('hello {}'.format(name))

greet('baihua')

### decorators that accept arguments or not arbitrarily
* the trouble is that decorator does not know whether it has been called with arguments
    * if not, func would be available at top-level, just need to return a decorated func
    * if yes, func would only be available at second-level, this is the exact same code as before
* 1) first method uses a boilerplate code that do the above logic by * syntax
* 2) second method is to use functools.partial() utility (see [Python cookbook](https://github.com/dabeaz/python-cookbook/blob/master/src/9/defining_a_decorator_that_takes_an_optional_argument/example.py) for details)

In [ ]:
""" 1) decorator that takes arguments arbitrarily """

import functools

def repeat(_func=None, *, num_times=2):
    """ repeat the decorated function by (otptional) num_times times; default=2 """
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for idx in range(num_times):
                print('repeating {}'.format(idx))
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    
    if _func is None:
        # if _func is None means decorator takes arguments
        # just return the inner decorator function
        return decorator_repeat
    else:
        # if _func is not None means decorator takes no arguments
        # returns _func decorated by the inner decorator function
        # note that in this case num_times would use default value
        # this is why * syntax is needed 
        # (* forces all subsequent to be keyword instead of positional arguments,
        # thus requiring a default value)
        return decorator_repeat(_func)

@repeat(num_times=5)
def say_whee():
    print('whee!')

@repeat
def say_hello():
    print('hello!')

say_whee()
say_hello()